In [1]:
import pandas as pd
import sys
import os

In [2]:
#import event and condition dataframes
event_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\event_df.csv")
conditon_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\condition_df.csv")

In [3]:
file_path = (r"C:\Users\Miranda\.ipython\CapstoneCoding\OutpatientVisitsdta\OutpatientVisits.dta")
outpatient_df = pd.read_stata(file_path)
outpatient_df.head(3)

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,FFEEIDX,PANEL,MPCDATA,OPDATEYR,OPDATEMM,...,OPDOF23X,OPDSL23X,OPDWC23X,OPDOT23X,OPDXP23X,OPDTC23X,IMPFLAG,PERWT23F,VARSTR,VARPSU
0,2790002,101,2790002101,2790002101001401,4 ROUND 4,-1,27 PANEL 27,1 HAS MPC DATA,2023,7,...,0.0,0.0,0.0,0.0,0.00,0.0,2 COMPLETE MPC DATA,11664.426815,2019,1
1,2790011,102,2790011102,2790011102006201,3 ROUND 3,-1,27 PANEL 27,2 NO MPC DATA,2023,1,...,0.0,0.0,0.0,0.0,121.10,180.0,1 COMPLETE HC DATA,10754.599229,2029,3
2,2790011,102,2790011102,2790011102006301,3 ROUND 3,-1,27 PANEL 27,2 NO MPC DATA,2023,1,...,0.0,0.0,0.0,0.0,380.32,1410.0,1 COMPLETE HC DATA,10754.599229,2029,3


In [4]:
outpatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20720 entries, 0 to 20719
Data columns (total 57 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   DUID            20720 non-null  int32   
 1   PID             20720 non-null  int16   
 2   DUPERSID        20720 non-null  object  
 3   EVNTIDX         20720 non-null  object  
 4   EVENTRN         20720 non-null  category
 5   FFEEIDX         20720 non-null  object  
 6   PANEL           20720 non-null  category
 7   MPCDATA         20720 non-null  category
 8   OPDATEYR        20720 non-null  category
 9   OPDATEMM        20720 non-null  category
 10  SEEDOC_M18      20720 non-null  category
 11  DRSPLTY_M18     20720 non-null  category
 12  MEDPTYPE_M18    20720 non-null  category
 13  VSTCTGRY        20720 non-null  category
 14  VSTRELCN_M18    20720 non-null  category
 15  LABTEST_M18     20720 non-null  category
 16  SONOGRAM_M18    20720 non-null  category
 17  XRAYS_M18   

In [5]:
outpatient_df = outpatient_df[['DUPERSID','OPDATEMM','OPDMD23X','OPDMR23X','OPDPV23X', 'EVNTIDX', 'EVENTRN',
            'OPFMD23X','OPFMR23X','OPFPV23X']].copy()

In [6]:
outpatient_df.rename(columns={'DUPERSID': 'person_id', 'EVNTIDX' : 'visit_id', 'EVENTRN' : 'round',
                      'OPDATEMM': 'month','OPDMD23X':'dr_medicaid',
                      'OPDMR23X': 'dr_medicare',
                      'OPDPV23X': 'dr_private','OPFMD23X':'f_medicaid','OPFMR23X': 'f_medicare',
                      'OPFPV23X' : 'f_private'}, inplace=True)
outpatient_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20720 entries, 0 to 20719
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   person_id    20720 non-null  object  
 1   month        20720 non-null  category
 2   dr_medicaid  20720 non-null  float64 
 3   dr_medicare  20720 non-null  float64 
 4   dr_private   20720 non-null  float64 
 5   visit_id     20720 non-null  object  
 6   round        20720 non-null  category
 7   f_medicaid   20720 non-null  float64 
 8   f_medicare   20720 non-null  float64 
 9   f_private    20720 non-null  float64 
dtypes: category(2), float64(6), object(2)
memory usage: 1.3+ MB


In [7]:
#df['sum_cols'] = df['col1'] + df['col2']
outpatient_df['medicaid'] = outpatient_df['dr_medicaid'] + outpatient_df['f_medicaid']
outpatient_df['medicare'] = outpatient_df['dr_medicare'] + outpatient_df['f_medicaid']
outpatient_df['private'] = outpatient_df['dr_private'] + outpatient_df['f_private']

#drop columns from df
outpatient_df = outpatient_df.drop(['dr_medicaid', 'dr_medicare', 'dr_private',
                   'f_medicare', 'f_medicaid', 'f_private'], axis=1)

In [8]:
#drop invalid months
outpatient_df = outpatient_df.dropna(subset=['month'])

#change month datatype
#category label to string, coerce to NANs
outpatient_df['month'] = pd.to_numeric(outpatient_df['month'].astype(str), errors='coerce')
#drop Nans
outpatient_df = outpatient_df.dropna(subset=['month'])
#change datatype to int
outpatient_df['month'] = outpatient_df['month'].astype(int)

#change visit id to int
outpatient_df['visit_id'] = outpatient_df['visit_id'].astype(int)

outpatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20720 entries, 0 to 20719
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   person_id  20720 non-null  object  
 1   month      20720 non-null  int64   
 2   visit_id   20720 non-null  int64   
 3   round      20720 non-null  category
 4   medicaid   20720 non-null  float64 
 5   medicare   20720 non-null  float64 
 6   private    20720 non-null  float64 
dtypes: category(1), float64(3), int64(2), object(1)
memory usage: 991.8+ KB


In [9]:
#import event_df and condition_df
event_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\event_df.csv")

condition_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\condition_df.csv")

In [10]:
#merge outpatient df with event next link medical condtion
outpatient_df = outpatient_df.merge(event_df, on='visit_id', how='left')

outpatient_df = outpatient_df.merge(condition_df, on='condition_id', how='left')

#check new hospital dataframe
outpatient_df.head()

,person_id,month,visit_id,round,medicaid,medicare,private,condition_id,ICD10
0,2790002101,7,2790002101001401,4 ROUND 4,0.0,0.00,127.42,2.790002e+12,L30
1,2790011102,1,2790011102006201,3 ROUND 3,0.0,121.10,0.00,2.790011e+12,-15
2,2790011102,1,2790011102006301,3 ROUND 3,0.0,210.86,169.46,2.790011e+12,-15
3,2790011102,1,2790011102006401,3 ROUND 3,0.0,0.00,0.00,2.790011e+12,I51
4,2790011102,1,2790011102006501,3 ROUND 3,0.0,0.00,0.00,NaN,NaN


In [11]:
#drop duplicates at visit and condition level together
outpatient_df = outpatient_df.drop_duplicates(subset=['visit_id','condition_id'])

#drop -15 from ICD10
outpatient_df = outpatient_df[outpatient_df['ICD10'] != '-15']

In [12]:
#outpatient dataframe
outpatient_df.head()

,person_id,month,visit_id,round,medicaid,medicare,private,condition_id,ICD10
0,2790002101,7,2790002101001401,4 ROUND 4,0.0,0.00,127.42,2.790002e+12,L30
3,2790011102,1,2790011102006401,3 ROUND 3,0.0,0.00,0.00,2.790011e+12,I51
4,2790011102,1,2790011102006501,3 ROUND 3,0.0,0.00,0.00,NaN,NaN
7,2790017101,2,2790017101003501,3 ROUND 3,0.0,0.00,342.00,NaN,NaN
8,2790019101,11,2790019101001801,5 ROUND 5,0.0,7.09,0.00,2.790019e+12,E11


In [13]:
#check the number of visits the outpatient entity had
outpatient_visits_2023 = outpatient_df['visit_id'].count()
#store variable for analysis notebook
%store outpatient_visits_2023
print(outpatient_visits_2023)

Stored 'outpatient_visits_2023' (int64)
20214


In [14]:
#count conditions per visit(eliminate double payments for one visit with multiple conditions)
condition_count = outpatient_df.groupby('visit_id')['condition_id'].nunique().reset_index(name='num_conditions')

#single visits by condition
single_condition = condition_count[condition_count['num_conditions'] ==1]['visit_id']
outpatient_df = outpatient_df[outpatient_df['visit_id'].isin(single_condition)].copy()

In [15]:
outpatient_df = outpatient_df[['ICD10', 'medicaid','medicare', 'private','month']]
outpatient_df.head(3)

,ICD10,medicaid,medicare,private,month
0,L30,0.0,0.00,127.42,7
3,I51,0.0,0.00,0.00,1
8,E11,0.0,7.09,0.00,11


In [16]:
#save outpatient dataframe to csv file for Tableau
outpatient_df.to_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\dataframes_csv_format\outpatient_df.csv", index=False)